In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd

import collections
import re
import pprint as pp
import numpy as np
import collections

import multiprocessing as mp
from multiprocessing.pool import ThreadPool

import math
import gzip
import pickle as pkl
from datetime import datetime
import matplotlib.pyplot as plt

import joblib


import fonctions
import itertools
from tqdm.notebook import tqdm

from os import listdir
from os.path import isfile, join
from collections import defaultdict

import random
random.seed(0)

Using TensorFlow backend.


In [2]:
def get_behavior(directory,doc_name,init):
    
    all_features = ["text_tokens", "hashtags", "tweet_id", 
                    "present_media", "present_links", 
                    "present_domains", "tweet_type","language", 
                    "tweet_timestamp", "engaged_with_user_id",
                    "engaged_with_user_follower_count", "engaged_with_user_following_count", 
                    "engaged_with_user_is_verified", "engaged_with_user_account_creation",
                    "engaging_user_id", "engaging_user_follower_count", 
                    "engaging_user_following_count", "engaging_user_is_verified",
                    "engaging_user_account_creation", "engagee_follows_engager"]
    
    df = pd.read_csv(directory+doc_name, encoding="utf-8", sep='\x01', header=None)
    
    if init==True:
        labels = ['reply_timestamp','retweet_timestamp', 'retweet_with_comment_timestamp','like_timestamp']
        all_variables = all_features + labels
        df.columns = all_variables
        
        df['reply_timestamp']=[ 0 if math.isnan(x) else 1 for x in df['reply_timestamp'] ]
        df['retweet_timestamp']=[ 0 if math.isnan(x) else 1 for x in df['retweet_timestamp'] ]
        df['retweet_with_comment_timestamp']=[ 0 if math.isnan(x) else 1 for x in df['retweet_with_comment_timestamp'] ]
        df['like_timestamp']=[ 0 if math.isnan(x) else 1 for x in df['like_timestamp'] ]
        
        df['tweet_timestamp']=[ str(datetime.utcfromtimestamp(int(date)))[0:10] for date in df['tweet_timestamp'] ]
        
        df = df.filter(labels+['tweet_timestamp','engaged_with_user_id', 'engaging_user_id'],axis=1)
        
    else:
        df.columns = all_features
        df['tweet_timestamp']=[ str(datetime.utcfromtimestamp(int(date)))[0:10] for date in df['tweet_timestamp'] ]
        df = df.filter(['tweet_timestamp','engaged_with_user_id', 'engaging_user_id'], axis=1)

    return df

def behavior_on_chunk(directory, chunk, chunk_id, dates, init):
    
    engagements = ['like_timestamp','retweet_timestamp','retweet_with_comment_timestamp','reply_timestamp']
    
    author_presence = {}
    buff_author_presence = {}

    iteration=1
    for batch_file in chunk:

        df = get_behavior(directory, batch_file, init)
  
        select = { k: [0,0,0,0,0,0,0] for k in np.unique(df.engaged_with_user_id) if k not in buff_author_presence.keys()  }
        buff_author_presence.update(select)

        for idx, date in enumerate(dates):       
            buff =  df[ df['tweet_timestamp']==date ]
            buff_presence = collections.Counter(buff.engaged_with_user_id)
            { update_date(buff_author_presence, k, v, idx) for k,v in buff_presence.items() }

        if iteration in [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550]:        
            select = {k:v for k,v in buff_author_presence.items() if k in author_presence.keys() }
            reste = {k:v for k,v in buff_author_presence.items() if k not in author_presence.keys() }
            { update_agg(author_presence, k, v) for k,v in select.items() }
            author_presence.update(reste)
            buff_author_presence={}
            print(len( author_presence.keys() ))

        print(iteration)
        iteration=iteration+1
        
    select = {k:v for k,v in buff_author_presence.items() if k in author_presence.keys() }
    reste = {k:v for k,v in buff_author_presence.items() if k not in author_presence.keys() }
    { update_agg(author_presence, k, v) for k,v in select.items() }
    author_presence.update(reste)
    
    print('cutting and saving...')

    if init==True:

        with gzip.open('/home/maxime/Desktop/RecSys2020/trends/author_presence_{}.pkl.gz'.format(chunk_id), 'wb') as f:
            pkl.dump(author_presence,f)

    else: 
        
        with gzip.open('/home/maxime/Desktop/RecSys2020/trends/update1_presence_metric_{}.pkl.gz'.format(chunk_id), 'wb') as f:
            pkl.dump(author_presence,f)
            
    return True

def update_date(author_presence, key, v, idx):
    author_presence[key][idx]=author_presence[key][idx]+v

def update_agg(author_presence, k, v):
    author_presence[k]= [x + y for x, y in zip(author_presence[k], v )]

In [10]:
batch_path='/home/maxime/Desktop/RecSys2020/data/batches'
batch_list = [f for f in listdir(batch_path) if isfile(join(batch_path, f))]
chunks = fonctions.chunkIt(batch_list,8)
directory = '/home/maxime/Desktop/RecSys2020/data/batches/'
dates = ['2020-02-06','2020-02-07', '2020-02-08','2020-02-09', '2020-02-10', '2020-02-11', '2020-02-12']

if __name__ == '__main__':
    
    # Setup a list of processes that we want to run
    processes = [ mp.Process(target=behavior_on_chunk, args=(directory, chunk, chunk_id, dates, True) ) for chunk_id, chunk in zip([4,5,6,7],[chunks[4],chunks[5],chunks[6],chunks[7]]) ]

    # Run processes
    for p in processes:
        p.start()
        
    #Stop the processes
    for p in processes:
        p.join() 
        

  

1
1
1
1
2
2
2
2
3
3
3
3
4
4
4
4
5
5
5
5
6
6
6
6
7
7
7
7
8
8
8
8
9
9
9
9
10
10
10
10
11
11
11
11
12
12
12
12
13
13
13
14
13
14
14
15
14
15
15
16
16
15
16
17
17
16
17
18
18
17
18
19
19
18
19
20
20
19
20
21
21
20
21
22
22
22
21
23
23
23
24
22
24
24
25
23
25
25
24
26
26
26
25
27
27
27
28
26
28
28
29
27
29
29
28
30
30
30
31
29
31
31
32
30
32
32
33
33
31
33
34
34
32
34
35
35
33
35
36
36
34
36
37
37
37
35
38
38
38
36
39
39
37
40
39
40
38
41
41
40
42
39
42
41
43
42
40
43
43
44
41
44
44
45
42
45
46
45
43
46
47
46
44
47
48
47
48
45
49
49
46
48
49
47
764889
50
765620
50
48
51
51
49
765666
50
52
52
51
53
766154
50
53
52
54
51
53
54
55
52
55
54
53
56
56
55
57
54
57
56
58
55
58
57
56
59
59
58
57
60
60
59
58
61
60
61
59
62
61
62
60
63
62
63
64
61
63
64
65
62
64
65
63
66
65
66
66
67
67
64
67
65
68
68
68
66
69
69
69
67
68
70
70
70
69
71
71
71
72
72
70
72
73
73
73
71
74
74
72
74
75
75
75
73
76
76
76
74
77
77
77
75
78
78
78
79
76
79
79
80
77
80
80
78
81
81
81
79
82
82
82
80
83
83
83
81
84
84
82
85
84
85


In [3]:
## update on validation

batch_path='/home/maxime/Desktop/RecSys2020/data/test'
batch_list = [f for f in listdir(batch_path) if isfile(join(batch_path, f))]
chunks = fonctions.chunkIt(batch_list, 8)
directory = '/home/maxime/Desktop/RecSys2020/data/test/'
# dates = ['2020-02-06','2020-02-07', '2020-02-08','2020-02-09', '2020-02-10', '2020-02-11', '2020-02-12']
dates = ['2020-02-13','2020-02-14', '2020-02-15','2020-02-16', '2020-02-17', '2020-02-18', '2020-02-19']

if __name__ == '__main__':
    # Setup a list of processes that we want to run
        
    processes = [ mp.Process(target=behavior_on_chunk, args=(directory, chunk, chunk_id, dates, False) ) for chunk_id, chunk in enumerate(chunks) ]

    # Run processes
    for p in processes:
        p.start()
        
    #Stop the processes
    for p in processes:
        p.join() 

1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
3
3
2
3
3
3
3
3
4
4
3
4
4
4
4
4
4
5
5
5
5
5
5
5
6
5
6
6
6
6
6
6
7
7
6
7
7
7
7
7
7
8
8
8
8
8
8
8
9
8
9
9
9
9
9
9
10
10
9
10
10
10
10
10
10
11
11
11
11
11
11
12
11
12
11
12
12
12
12
13
12
13
12
13
13
13
13
13
14
14
13
14
14
14
14
15
14
15
14
15
15
15
15
15
16
16
15
16
16
16
16
17
17
16
17
16
17
17
17
17
18
18
17
18
18
18
19
19
18
18
18
19
19
19
20
20
19
19
19
20
20
20
21
21
20
20
20
21
21
21
22
21
22
21
21
22
22
22
23
22
23
22
22
23
23
23
24
23
23
23
24
24
24
24
24
25
24
24
25
25
25
25
25
25
26
26
25
26
26
26
26
26
27
26
27
27
27
27
27
27
27
28
28
28
28
28
28
28
28
29
29
29
29
29
30
29
29
29
30
30
30
30
31
30
30
30
31
31
31
31
31
32
31
31
32
32
32
32
32
33
32
32
33
33
33
33
33
33
34
34
33
34
34
34
34
34
35
34
35
35
35
35
36
35
35
35
36
36
37
36
36
36
36
36
37
37
37
38
37
37
37
37
38
38
39
38
38
38
38
38
39
39
39
39
40
39
39
40
39
40
40
41
40
40
40
40
41
41
42
41
41
41
41
41
42
42
42
42
42
43
42
42
43
43
44
43
43
43
43
43
45
44
44
44
44
44
44
44
45
46
45
45
4